In [1]:
import pandas as pd
import spacy
import requests
from bs4 import BeautifulSoup
import PyPDF2
from neo4j import GraphDatabase
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain.graphs import Neo4jGraph
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain_community.llms import Ollama
from langchain.text_splitter import TokenTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector

from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough
from langchain.docstore.document import Document 
'''part of the LangChain library and is used to represent a piece of text or document data along with its metadata. 
It is a common data structure used throughout the LangChain library for storing and processing text data.
In your code, you are using the Document class to create new Document objects from the preprocessed text data. 
Specifically, you are creating a dictionary with the page_content (the actual text content) and metadata (additional information about the text, such as the source), 
and then creating a Document object using that dictionary.'''
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from py2neo import Graph
from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings import OpenAIEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser
from pydantic import BaseModel
from typing import List


In [2]:
URI = "bolt://localhost:7687"
graph = Neo4jGraph(url=URI, username="neo4j", password="password")

from langchain_community.document_loaders import TextLoader
loader = TextLoader("hello.txt")
raw_documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

# Preprocess the documents to convert lists to tuples
'''we create a new dictionary with the page_content and metadata keys, where the metadata key has a nested dictionary with the source key. 
We convert the source value to a tuple if it's a list, and then create a new Document object from this dictionary using the Document constructor.
This way, we avoid using the non-existent to_dict and from_dict methods and create a new Document object with the desired structure.
With this modification, the code should run without the AttributeError: 'Document' object has no attribute 'to_dict' error.'''
preprocessed_documents = []
for doc in documents:
    data = {
        "page_content": doc.page_content,
        "metadata": {
            "source": tuple(doc.metadata["source"]) if isinstance(doc.metadata["source"], list) else doc.metadata["source"]
        }
    }
    preprocessed_documents.append(Document(**data))


llm = OllamaFunctions(model="llama3")
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(preprocessed_documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [4]:
!pip install neo4j

Defaulting to user installation because normal site-packages is not writeable


In [15]:


# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en-v1.5"
# )
from langchain_community.embeddings import OllamaEmbeddings


embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url="bolt://localhost:7687",
    username='neo4j',
    password='password',
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
# Retriever
from py2neo import                        Graph
graph=Graph('bolt://localhost:7687', name='neo4j')

graph.run(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]"
)

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)



In [16]:
from pyvis.network import Network

def showGraph(graph_documents):
    net = Network(height="800px", width="100%", bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships
    
    for node in nodes:
        net.add_node(node.id, label=node.id, title=str(node.type), color="skyblue")
    
    for relationship in relationships:
        net.add_edge(relationship.source.id, relationship.target.id, title=relationship.type, color="gray", arrows="to")
    
    net.repulsion()
    
    # Generate HTML file
    net.write_html("graph.html")

In [17]:
showGraph(graph_documents)

In [18]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [19]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3", format="json", temperature=0)

In [20]:
response = vector_index.similarity_search(
    "tell me about queen elizabeth"
)
print(response[0])

page_content='\ntext:  evidence against her. Instead, on 22 May, Elizabeth was moved from the Tower to Woodstock, where she was to spend almost a year under house arrest in the charge of Henry Bedingfeld. Crowds cheered her all along the way.[37][e]\n\nOn 17 April 1555, Elizabeth was recalled to court to attend the final stages of Mary\'s apparent pregnancy. If Mary and her child died, Elizabeth would become queen, but if Mary gave birth to a healthy child, Elizabeth\'s chances of becoming queen would recede sharply. When it became clear that Mary was not pregnant, no one believed any longer that she could have a child.[39] Elizabeth\'s succession seemed assured.[40]\n\nKing Philip, who ascended the Spanish throne in 1556, acknowledged the new political reality and cultivated his sister-in-law. She was a better ally than the chief alternative, Mary, Queen of Scots, who had grown up in France and was betrothed to the Dauphin of France.[41] When his wife fell ill in 1558, Philip sent the

In [21]:
from langchain.chains import RetrievalQA

vector_qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)
vector_qa.run(
    "when was queen elizath born "
)

/home/asura/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'{ "text": "Queen Elizabeth I of England was born on September 7, 1533." }'

In [22]:
vector_qa.run(
    "when did elizabeth become queen"
)

'{ "Elizabeth I (7 September 1533 – 24 March 1603)[a] was Queen of England and Ireland from 17 November 1558 until her death in 1603." \n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n    \n\n\n\n\n\n'

In [23]:
vector_qa.run(
    "what is google"
)

'{ }'

In [24]:
vector_qa.run("Tell me about Queeen Elizabeth.")

'{ }'